## Imports

In [ ]:
from nb_utils import set_root
PROJECT_DIR = set_root(3)
print("PROJECT_DIR: " + str(PROJECT_DIR))

In [ ]:
from mlpr.ml.supervisioned.regression import metrics, plots
from mlpr.ml.tunning.grid_search import GridSearch
from mlpr.reports.create import ReportGenerator

from utils.reader import read_file_yaml

import pandas as pd
import numpy as np
import sklearn.datasets as sk_datasets
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

%reload_ext autoreload

## Parameters

In [ ]:
path_data = PROJECT_DIR  / 'data'
path_conf = PROJECT_DIR  / 'conf'
path_templates = PROJECT_DIR / 'templates'
path_assets = PROJECT_DIR / 'assets'

path_intermediate = path_data / '02_intermediate'
path_models = path_data / '04_models'
path_reports = path_data / '05_reports'

path_reports_regression = path_templates / "regression"
file_path_reports = path_reports / "report_model.html"

parameters = read_file_yaml(path_conf / 'params.yaml')

datasets = parameters["datasets"]
n_seed = parameters["experiments"]["n_seed"]

In [ ]:
np.random.seed(n_seed)

## Read

In [ ]:
content = getattr(sk_datasets, datasets["diabetes"]["path"])()
data = pd.DataFrame(content["data"], columns=content["feature_names"])
data["target"] = content["target"]

## Split labels

In [ ]:
X = data.drop("target", axis=1)
y = data["target"].values

## Modelling

In [ ]:
models_params = {
    Ridge: {'alpha': [0.1, 1.0, 10.0], 'random_state': [n_seed]},
    Lasso: {'alpha': [0.1, 1.0, 10.0], 'random_state': [n_seed]},
    SVR: {'C': [0.1, 1.0, 10.0], 'kernel': ['linear', 'rbf']},
    RandomForestRegressor: {'n_estimators': [10, 50, 100], 'max_depth': [None, 5, 10], 'random_state': [n_seed]},
    GradientBoostingRegressor: {'n_estimators': [100, 200], 'learning_rate': [0.1, 0.05, 0.01], 'random_state': [n_seed]},
    XGBRegressor: {'n_estimators': [100, 200], 'learning_rate': [0.1, 0.05, 0.01], 'random_state': [n_seed]}
}

params_split = {'test_size': 0.25, 'random_state': n_seed}
params_norm = {'with_mean': True, 'with_std': True}

grid_search = GridSearch(
    X,
    y,
    params_split=params_split,
    models_params=models_params,
    normalize=True,
    params_norm=params_norm
    )
grid_search.search()

best_model, best_params = \
    grid_search \
        .get_best_model()


## Predictions

In [ ]:
data_test = pd.DataFrame(
    grid_search.X_train,
    columns=X.columns
)
data_test["y_true"] = grid_search.y_train
data_test["y_pred"] = grid_search.best_model.predict(grid_search.X_train)

In [ ]:
rp = plots.RegressionPlots(data_test, color_palette=["#FF4B3E", "#1C2127"])

In [ ]:
fig, axs = rp.grid_plot(
    plot_functions=[
        ['scatter', 'plot_ecdf'],
        ['plot_kde', 'plot_error_hist']
    ],
    plot_args={
        'scatter': {'y_true_col': 'y_true', 'y_pred_col': 'y_pred', 'linecolor': '#1C2127'},
        'plot_ecdf': {'y_true_col': 'y_true', 'y_pred_col': 'y_pred'},
        'plot_kde': {'columns': ['y_true', 'y_pred']},
        'plot_error_hist': {'y_true_col': 'y_true', 'y_pred_col': 'y_pred', 'linecolor': '#1C2127'}
    },
    show_inline=True
)

## Metrics

In [ ]:
k = 4
rm = metrics.RegressionMetrics(data_test, *["y_true", "y_pred"])

In [ ]:
results = rm.calculate_metrics(
    ["mape", "rmse", "kolmogorov_smirnov", "confusion_matrix", "calculate_kappa"],
    {
        "mape": {},
        "rmse": {},
        "kolmogorov_smirnov": {},
        "confusion_matrix": {"n_bins": k},
        "calculate_kappa": {"n_bins": k}
    }
)

## Save image

In [ ]:
fig.savefig(
    path_assets / "regression_plots.png",
    dpi=400,
    bbox_inches='tight'
)

## Reports

In [ ]:
report = ReportGenerator(template_dir = path_reports_regression)
report.render(
    metrics=results,
    template_file="template_v3.html",
    output_file=file_path_reports,
    image_path=path_intermediate / "regression_plots.png"
)

In [ ]:
results